In [0]:
######################### load dependencies #########################
import os
import io
import re
import warnings
import nltk
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import svm


from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from google.colab import files


In [0]:
from sklearn.svm import LinearSVC

In [0]:
from sklearn.naive_bayes import MultinomialNB

In [0]:
from sklearn.ensemble import VotingClassifier

In [0]:
from scipy.stats import mode

In [0]:
######################### upload dataset #########################
uploaded = files.upload()

Saving IMDB Dataset.csv to IMDB Dataset.csv


In [0]:
######################### load data #########################
# import data
imdb_data = pd.read_csv(io.BytesIO(uploaded['IMDB Dataset.csv']))

# data description
imdb_data.describe()

# value counts
imdb_data['sentiment'].value_counts()

# print head (top 5 rows)
imdb_data.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [0]:
# encode labels ["positive", "negative"] to [0,1]
le = LabelEncoder()
imdb_data['sentiment'] = le.fit_transform(imdb_data.sentiment)

In [0]:
#########################  Split the dataset in the proportion of 4:1  #########################
# i.e 80% training data and 20% testing data
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(imdb_data['review'], imdb_data['sentiment'], test_size=0.2)
# dimensions of training data
print(X_train.shape, y_train.shape)
# dimensions of testing data
print(X_test.shape, y_test.shape)

(40000,) (40000,)
(10000,) (10000,)


In [0]:
#########################  Clean Data and preprocess  #########################
# Tokenization of text
tokenizer = ToktokTokenizer()
# Setting English stopwords
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()


# Removing the square brackets


def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)


# Removing the noisy text


def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

# Apply function on review column


imdb_data['review'] = imdb_data['review'].apply(denoise_text)

In [0]:
def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', text)
    return text

# Apply function on review column
imdb_data['review'] = imdb_data['review'].apply(remove_special_characters)

In [0]:
# Stemming the text
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

# Apply function on review column
imdb_data['review'] = imdb_data['review'].apply(simple_stemmer)

In [0]:
# set stopwords to english
stop = set(stopwords.words('english'))
print(stop)

# removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

# Apply function on review column
imdb_data['review'] = imdb_data['review'].apply(remove_stopwords)

{'aren', 'those', 'further', "hasn't", 'for', 'so', 'his', 'me', 'myself', 'once', 'couldn', 'had', 'while', 'than', 'through', 'how', "hadn't", 'who', "mustn't", "you'd", 'no', 'it', 'same', 'll', 'mightn', 'themselves', 'any', 'its', 'hadn', 'doesn', 'some', 'above', 'do', 'not', 'this', 'after', 'weren', 'been', "doesn't", 'then', 've', 'be', 'ours', 'between', 'haven', 'just', 'mustn', 'them', 'during', 'more', "aren't", 'too', 'himself', 'down', 'am', "you're", 'in', 'd', "haven't", 'theirs', 'what', 'when', 'shan', 'm', 'a', 'is', 'yourself', "wouldn't", 'can', 'again', 'yours', 'an', 'nor', 'other', 'shouldn', 'from', 'of', 'very', 'here', 'hers', 'does', 'by', 'didn', 'now', 'herself', 'and', 'yourselves', "weren't", 'until', 'on', 'most', 'out', 'don', "isn't", 'only', 'such', 'these', "won't", 't', 'did', 'ourselves', 'own', 'below', 'their', "don't", 'should', 'won', 'about', 'y', 'off', 'they', 'where', "shouldn't", 'him', 'itself', 'which', 're', 'we', 'into', "couldn't", 

######################### Preprocessing #########################

> Indented block




In [0]:
######################### Countvectarization(with monograms) #########################
# if this has an error remove "min_df=0,max_df=1,binary=False"
basic_cv = CountVectorizer(
    stop_words='english', min_df=0,max_df=1,binary=False)
# transformed train reviews
basic_cv_train_reviews = basic_cv.fit_transform(X_train)
# transformed test reviews
basic_cv_test_reviews = basic_cv.transform(X_test)

In [0]:
######################### Countvectarization(with n-grams) #########################
ngrams_cv=CountVectorizer(
    stop_words='english', min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
ngrams_cv_train_reviews=ngrams_cv.fit_transform(X_train)
#transformed test reviews
ngrams_cv_test_reviews=ngrams_cv.transform(X_test)

In [0]:
######################### TfidfVectorization(with monograms) #########################
basic_tfidf = TfidfVectorizer(max_df=0.5, min_df=2,
                                   stop_words='english')
# transformed train reviews
basic_tf_train_reviews = basic_tfidf.fit_transform(X_train)
# transformed test reviews
basic_tf_test_reviews = basic_tfidf.transform(X_test)

In [0]:
######################### TfidfVectorization(with n-grams) #########################
ngrams_tfidf = TfidfVectorizer(max_df=0.5, min_df=2,
                                   stop_words='english', ngram_range=(1,3))
# transformed train reviews
ngrams_tf_train_reviews = ngrams_tfidf.fit_transform(X_train)
# transformed test reviews
ngrams_tf_test_reviews = ngrams_tfidf.transform(X_test)

######################### Logistic Regression #########################


In [0]:
######################### Logistic Regression with Count Vectorization (mono) #########################
# training the model
lr_base_cv = LogisticRegression(penalty='l2', solver="sag", random_state=0, max_iter=2000)
# Fitting the model 
lr_bow = lr_base_cv.fit(basic_cv_train_reviews, y_train)

# Predicting the model 
lr_bow_predict=lr_base_cv.predict(basic_cv_test_reviews)
print(lr_bow_predict)
# Accuracy score
lr_accuracy = accuracy_score(y_test, lr_bow_predict)
print("lr_accuracy :", lr_accuracy)
# Classification report
lr_bow_report = classification_report(y_test, lr_bow_predict)
print(lr_bow_report)

[0 0 0 ... 0 0 0]
lr_accuracy : 0.5354
              precision    recall  f1-score   support

           0       0.52      0.87      0.65      4970
           1       0.62      0.20      0.31      5030

    accuracy                           0.54     10000
   macro avg       0.57      0.54      0.48     10000
weighted avg       0.57      0.54      0.48     10000



In [0]:
######################### Logistic Regression with Count Vectorization (n grams) #########################
# training the model
lr = LogisticRegression(penalty='l2', solver="sag", random_state=0, max_iter=2000)
# Fitting the model 
lr_ngram = lr.fit(ngrams_cv_train_reviews, y_train)

# Predicting the model 
lr_ngram_predict=lr.predict(ngrams_cv_test_reviews)
print(lr_ngram_predict)
# Accuracy score
lr_ngram_score = accuracy_score(y_test, lr_ngram_predict)
print("lr_ngram_score :", lr_ngram_score)
# Classification report
lr_ngram_report = classification_report(y_test, lr_ngram_predict)
print(lr_ngram_report)


[0 0 0 ... 0 0 0]
lr_ngram_score : 0.5838
              precision    recall  f1-score   support

           0       0.55      0.98      0.70      4970
           1       0.91      0.19      0.32      5030

    accuracy                           0.58     10000
   macro avg       0.73      0.59      0.51     10000
weighted avg       0.73      0.58      0.51     10000



In [0]:
######################### Logistic Regression with Tf-IDF (mono grams) #########################
# training the model
lr_base_tfid = LogisticRegression(penalty='l2', solver="sag", random_state=0, max_iter=2000)
# Fitting the model 
lr_basictf = lr_base_tfid.fit(basic_tf_train_reviews, y_train)

# Predicting the model 
lr_basictf_predict=lr_base_tfid.predict(basic_tf_test_reviews)
print(lr_basictf_predict)
# Accuracy score
lr_basictf_score = accuracy_score(y_test, lr_basictf_predict)
print("lr_basictf_score :", lr_basictf_score)
# Classification report
lr_basictf_report = classification_report(y_test, lr_basictf_predict)
print(lr_basictf_report)


[0 0 1 ... 0 0 1]
lr_basictf_score : 0.8949
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      4970
           1       0.89      0.91      0.90      5030

    accuracy                           0.89     10000
   macro avg       0.90      0.89      0.89     10000
weighted avg       0.90      0.89      0.89     10000



In [0]:
######################### Logistic Regression with Tf-IDF (with n grams) #########################
# training the model
lr_ngrams_tf = LogisticRegression(penalty='l2', solver="sag", random_state=0, max_iter=2000)
# Fitting the model 
lr_ngramtf = lr_ngrams_tf.fit(ngrams_tf_train_reviews, y_train)

# Predicting the model 
lr_ngramtf_predict=lr_ngrams_tf.predict(ngrams_tf_test_reviews)
print(lr_ngramtf_predict)
# Accuracy score
lr_ngramtf_score = accuracy_score(y_test, lr_ngramtf_predict)
print("lr_ngramtf_score :", lr_ngramtf_score)
# Classification report
lr_ngramtf_report = classification_report(y_test, lr_ngramtf_predict)
print(lr_ngramtf_report)


[0 0 1 ... 0 0 1]
lr_ngramtf_score : 0.8936
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      4970
           1       0.89      0.90      0.90      5030

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



######################### Naive Bayes #########################

In [0]:
######################### Naive Bayes with Count vectorization (with mono grams) #########################

nb_cv = MultinomialNB()
nb_cv.fit(basic_cv_train_reviews, y_train)
prediction_nb_cv = nb_cv.predict(basic_cv_test_reviews)

# accuracy score
base_cv_nb = accuracy_score(y_test, prediction_nb_cv)
print("nb_base_cv_accuracy :", base_cv_nb)
# report
base_cv_nb_report = classification_report(y_test, prediction_nb_cv)
print(base_cv_nb_report)

nb_base_cv_accuracy : 0.5527
              precision    recall  f1-score   support

           0       0.53      0.89      0.67      4998
           1       0.66      0.22      0.33      5002

    accuracy                           0.55     10000
   macro avg       0.60      0.55      0.50     10000
weighted avg       0.60      0.55      0.50     10000



In [0]:
######################### Naive Bayes with Count Vectorization (with n grams) #########################
nb_cvn = MultinomialNB()
nb_cvn.fit(ngrams_cv_train_reviews, y_train)
prediction_nb_cvn = nb_cvn.predict(ngrams_cv_test_reviews)

# accuracy score
ngrams_cv_nb = accuracy_score(y_test, prediction_nb_cvn)
print("nb_cvn_accuracy :", ngrams_cv_nb)

# report
ngrams_cv_nb_report = classification_report(y_test, prediction_nb_cvn)
print(ngrams_cv_svm_report)

nb_cvn_accuracy : 0.7435
              precision    recall  f1-score   support

           0       0.50      1.00      0.67      4998
           1       0.95      0.01      0.02      5002

    accuracy                           0.51     10000
   macro avg       0.73      0.51      0.35     10000
weighted avg       0.73      0.51      0.35     10000



In [0]:
######################### Naive Bayes with Tf-IDF (with mono grams) #########################
nb_tf = MultinomialNB()
nb_tf.fit(basic_tf_train_reviews, y_train)
prediction_nb_tf = nb_tf.predict(basic_tf_test_reviews)

# accuracy score
base_tfid_cv_nb = accuracy_score(y_test, prediction_nb_tf)
print("nb_tfid_n_accuracy :", base_tfid_cv_nb)

# report
report_ngrams_tfid_nb = classification_report(y_test, prediction_nb_tf)
print(report_ngrams_tfid_nb)

nb_tfid_n_accuracy : 0.8666
              precision    recall  f1-score   support

           0       0.86      0.87      0.87      4998
           1       0.87      0.86      0.87      5002

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



In [0]:
######################### Naive Bayes with Tf-IDF (with n grams) #########################
nb_tfn = MultinomialNB()
nb_tfn.fit(ngrams_tf_train_reviews, y_train)
prediction_nb_tfn = nb_tfn.predict(ngrams_tf_test_reviews)

# accuracy score
ngrams_tfid_cv_nb = accuracy_score(y_test, prediction_nb_tfn)
print("nb_tfid_base_accuracy :", ngrams_tfid_cv_nb)

# report
report_ngrams_tfid_nb = classification_report(y_test, prediction_nb_tfn)
print(report_ngrams_tfid_nb)

nb_tfid_base_accuracy : 0.8863
              precision    recall  f1-score   support

           0       0.89      0.88      0.89      4998
           1       0.88      0.89      0.89      5002

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



######################### Support Vector Machines #########################

In [0]:
######################### Support Vector Machines with Count Vectorization (with monogram) #########################

# Perform classification with SVM, kernel=linear
classifier_linear_cv = LinearSVC(random_state=0)
classifier_linear_cv.fit(basic_cv_train_reviews, y_train)
prediction_linear_cv = classifier_linear_cv.predict(basic_cv_test_reviews)

# accuracy score
base_cv_svm = accuracy_score(y_test, prediction_linear_cv)
print("base_cv_svm_accuracy :", base_cv_svm)
# report
base_cv_svm_report = classification_report(y_test, prediction_linear_cv)
print(base_cv_svm_report)

base_cv_svm_accuracy : 0.5492
              precision    recall  f1-score   support

           0       0.53      0.88      0.66      4998
           1       0.64      0.22      0.33      5002

    accuracy                           0.55     10000
   macro avg       0.59      0.55      0.50     10000
weighted avg       0.59      0.55      0.49     10000



In [0]:
######################### Support Vector Machines with Count Vectorization (with n grams) #########################

# Perform classification with SVM, kernel=linear
classifier_linear_cvn = LinearSVC(random_state=0)
classifier_linear_cvn.fit(ngrams_cv_train_reviews, y_train)
prediction_linear_cvn = classifier_linear_cvn.predict(ngrams_cv_test_reviews)

# accuracy score
ngrams_cv_svm = accuracy_score(y_test, prediction_linear_cvn)
print("prediction_linear_cvn_accuracy :", ngrams_cv_svm)

# report
ngrams_cv_svm_report = classification_report(y_test, prediction_linear_cvn)
print(ngrams_cv_svm_report)

prediction_linear_cvn_accuracy : 0.5065
              precision    recall  f1-score   support

           0       0.50      1.00      0.67      4998
           1       0.92      0.01      0.03      5002

    accuracy                           0.51     10000
   macro avg       0.71      0.51      0.35     10000
weighted avg       0.71      0.51      0.35     10000



In [0]:
######################### Support Vector Machines with Tf-IDF (with monogram) #########################

# Perform classification with SVM, kernel=linear
classifier_linear_tf = LinearSVC(random_state=0)
classifier_linear_tf.fit(basic_tf_train_reviews, y_train)
prediction_linear_tf = classifier_linear_tf.predict(basic_tf_test_reviews)

# accuracy score
base_tfid_cv_svm = accuracy_score(y_test, prediction_linear_tf)
print("prediction_linear_cvn_accuracy :", base_tfid_cv_svm)

# report
report_base_tfid_svm = classification_report(y_test, prediction_linear_tf)
print(report_base_tfid_svm)

prediction_linear_cvn_accuracy : 0.8937
              precision    recall  f1-score   support

           0       0.90      0.89      0.89      4998
           1       0.89      0.90      0.89      5002

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [0]:
######################### Support Vector Machines with Tf-IDF (with n grams) #########################

# Perform classification with SVM, kernel=linear
classifier_linear_tfn = LinearSVC(random_state=0)
classifier_linear_tfn.fit(ngrams_tf_train_reviews, y_train)
prediction_linear_tfn = classifier_linear_tfn.predict(ngrams_tf_test_reviews)

# accuracy score
# accuracy score
ngrams_tfid_cv_svm = accuracy_score(y_test, prediction_linear_tfn)
print("prediction_linear_cvn_accuracy :", ngrams_tfid_cv_svm)

# report
report_ngrams_tfid_svm = classification_report(y_test, prediction_linear_tfn)
print(report_ngrams_tfid_svm)

prediction_linear_cvn_accuracy : 0.9078
              precision    recall  f1-score   support

           0       0.92      0.90      0.91      4998
           1       0.90      0.92      0.91      5002

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000



######################### Blending the algorihms #########################
An approach named "voting" is used:


*   https://www-sciencedirect-com.libproxy.uregina.ca/science/article/pii/S1568494616305919
*   http://rasbt.github.io/mlxtend/user_guide/classifier/EnsembleVoteClassifier/#example-1-classifying-iris-flowers-using-different-classification-models



In [0]:
#################### Blending SVM + LR + NB ####################
def blend_using_voting_classifier(X_train, y_train, X_test, y_test):
    lsvc = LinearSVC(random_state=0)
    mnb = MultinomialNB()
    lr = LogisticRegression(penalty='l2', solver="sag", random_state=0, max_iter=2000)

    lr.fit(X_train, y_train)
    lsvc.fit(X_train, y_train)
    mnb.fit(X_train, y_train)

    pred1 = lsvc.predict(X_test)
    pred2 = mnb.predict(X_test)
    pred3 = lr.predict(X_test)

    final_pred = np.array([])
    for i in range(0,len(y_test)):
        final_pred = np.append(final_pred, mode([pred1[i], pred2[i], pred3[i]])[0][0])

    # accuracy score
    blend_acc = accuracy_score(y_test, final_pred)
    print("blend_acc :", blend_acc)

    # report
    blend_report = classification_report(y_test, final_pred)
    print(blend_report)

In [0]:
################### Blending (countvectorization - monograms) ###################
blend_using_voting_classifier(basic_cv_train_reviews, y_train, basic_cv_test_reviews, y_test)

blend_acc : 0.5354
              precision    recall  f1-score   support

           0       0.52      0.87      0.65      4970
           1       0.62      0.20      0.31      5030

    accuracy                           0.54     10000
   macro avg       0.57      0.54      0.48     10000
weighted avg       0.57      0.54      0.48     10000



In [0]:
################### Blending (countvectorization - ngrams) ###################
blend_using_voting_classifier(ngrams_cv_train_reviews, y_train, ngrams_cv_test_reviews, y_test)

blend_acc : 0.5835
              precision    recall  f1-score   support

           0       0.54      0.98      0.70      4970
           1       0.92      0.19      0.31      5030

    accuracy                           0.58     10000
   macro avg       0.73      0.59      0.51     10000
weighted avg       0.73      0.58      0.51     10000



In [0]:
################### Blending (Tf-IDF - monograms) ###################
blend_using_voting_classifier(basic_tf_train_reviews, y_train, basic_tf_test_reviews, y_test)

blend_acc : 0.8985
              precision    recall  f1-score   support

           0       0.90      0.89      0.90      4970
           1       0.90      0.90      0.90      5030

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



In [0]:
################### Blending (Tf-IDF - n-grams) ###################
blend_using_voting_classifier(ngrams_tf_train_reviews, y_train, ngrams_tf_test_reviews, y_test)

blend_acc : 0.9
              precision    recall  f1-score   support

           0       0.91      0.89      0.90      4970
           1       0.90      0.91      0.90      5030

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000

